<a href="https://colab.research.google.com/github/sapperka/AZSII/blob/main/%D0%90%D0%97%D0%A1%D0%98%D0%98_%D0%91%D0%91%D0%9C%D0%9E_02_23_%D0%A0%D1%8B%D0%B1%D0%B0%D0%BA%D0%BE%D0%B2_%D0%9F%D0%A06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# АЗСИИ

## Практика 6: Атака по переносу (Transfer Attack) на модели ИИ

Группа: ББМО-02-23

Студент: Рыбаков Владислав Андреевич

## Цель задания:
Изучить концепцию атаки по переносу, где противоречивые примеры, созданные для одной модели,
используются для атаки на другую модель. Это задание требует создания нескольких моделей,
генерации противоречивых примеров для одной модели и проверки их на другой модели.

## Задачи:
1. Загрузить несколько моделей, обученных на датасете MNIST.
2. Изучить теоретические основы атаки по переносу.
3. Реализовать атаку FGSM на одну модель и проверить, как противоречивые примеры влияют на другую модель.
4. Оценить точность обеих моделей на противоречивых примерах и проанализировать
переносимость атак.

## Шаги выполнения:

### Шаг 1: Загрузка и создание двух различных моделей

In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Модель 1: Простая полносвязная нейронная сеть
model1 = Sequential([
  Flatten(input_shape=(28, 28)),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')
])
# Компиляция модели
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics= ['accuracy'])
# Обучение модели
model1.fit(train_images, train_labels, epochs=5)
# Сохранение модели
model1.save('mnist_model1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8776 - loss: 0.4321
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9658 - loss: 0.1197
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9763 - loss: 0.0792
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.9841 - loss: 0.0534
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9875 - loss: 0.0409


In [4]:
# Модель 2: Свёрточная нейронная сеть (CNN)
model2 = Sequential([
  Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
  MaxPooling2D((2, 2)),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax')
])
# Компиляция модели
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=
['accuracy'])
# Обучение модели
model2.fit(train_images.reshape(-1, 28, 28, 1), train_labels, epochs=5)
# Сохранение модели
model2.save('mnist_model2.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9089 - loss: 0.3004
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 21ms/step - accuracy: 0.9828 - loss: 0.0538
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - accuracy: 0.9910 - loss: 0.0300
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - accuracy: 0.9939 - loss: 0.0206
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - accuracy: 0.9950 - loss: 0.0142


### Шаг 2: Теория атаки по переносу

Атака по переносу — это метод, при котором противоречивые примеры, созданные для одной
модели, применяются к другой модели. Это возможно, потому что модели часто имеют схожие
слабости и могут совершать одинаковые ошибки на определенных данных, даже если они обучены
на разных архитектурах.

В этом задании мы создадим противоречивые примеры для первой модели с помощью FGSM и
затем проверим, насколько эти примеры могут атаковать вторую модель.

### Шаг 3: Реализация атаки FGSM на первую модель

In [14]:
import numpy as np
# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  # Применение знака градиента к изображению
  perturbed_image = image + epsilon * np.sign(gradient)
  perturbed_image = np.clip(perturbed_image, 0, 1) # Убедиться, что значения остаются в пределах [0, 1]
  return perturbed_image

# Генерация противоречивых примеров для первой модели
def generate_fgsm_adversarial(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)))
        label = labels[i]

        # Вычисление градиента
        if len(label.shape) > 1 and label.shape[1] > 1:
            label = np.argmax(label)
        label = tf.convert_to_tensor(label)
        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)
        gradient = tape.gradient(loss, image)
        # Применяем FGSM:
        adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
        adversarial_images.append(np.reshape(adversarial_image, (28, 28, 1)))  # Сохраняем результат в массив
    return np.array(adversarial_images)

# Генерация противоречивых примеров для первой модели
epsilon = 0.1
adversarial_images_model1 = generate_fgsm_adversarial(model1, test_images, test_labels, epsilon)

### Шаг 4: Оценка противоречивых примеров на обеих моделях

In [15]:
# Оценка первой модели на противоречивых примерах
test_labels_argmax = np.argmax(test_labels, axis=1) # Преобразование onehot меток в целые числа
loss1, acc1 = model1.evaluate(adversarial_images_model1, test_labels)
print(f'Accuracy of model1 on adversarial examples: {acc1}')
# Оценка второй модели на противоречивых примерах (перенос атаки)
adversarial_images_model1_reshaped = adversarial_images_model1.reshape(-1, 28, 28, 1)
loss2, acc2 = model2.evaluate(adversarial_images_model1_reshaped, test_labels)
print(f'Accuracy of model2 on adversarial examples from model1: {acc2}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0709 - loss: 6.5701
Accuracy of model1 on adversarial examples: 0.09459999948740005
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9594 - loss: 0.1510
Accuracy of model2 on adversarial examples from model1: 0.9642999768257141


### Шаг 5: Анализ переносимости атак

In [16]:
# Генерация противоречивых примеров для второй модели
adversarial_images_model2 = generate_fgsm_adversarial(model2, test_images, test_labels, epsilon)

# Оценка второй модели на собственных противоречивых примерах
loss2_self, acc2_self = model2.evaluate(adversarial_images_model2, test_labels)
print(f'Accuracy of model2 on its own adversarial examples: {acc2_self}')

# Оценка первой модели на противоречивых примерах второй модели (перенос атаки)
adversarial_images_model2_reshaped = adversarial_images_model2.reshape(-1, 28, 28, 1)
loss1_transfer, acc1_transfer = model1.evaluate(adversarial_images_model2_reshaped, test_labels)
print(f'Accuracy of model1 on adversarial examples from model2: {acc1_transfer}')

# Оценка уязвимости и предложения
if acc2_self < acc1_transfer:
    print("Model2 is more vulnerable to FGSM attack on its own data compared to model1 under transfer attack.")
else:
    print("Model1 is more vulnerable under transfer attack from model2.")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6097 - loss: 1.7007
Accuracy of model2 on its own adversarial examples: 0.626800000667572
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9222 - loss: 0.2499
Accuracy of model1 on adversarial examples from model2: 0.9336000084877014
Model2 is more vulnerable to FGSM attack on its own data compared to model1 under transfer attack.


#### Вывод:
Атака FGSM оказала заметное влияние на точность модели 2, которая снизилась до 62.7% на собственных противоречивых примерах. Однако атака по переносу на модель 1 оказалась менее эффективной, и модель 1 сохранила высокую точность 93.4%. Это указывает на то, что модель 2 более уязвима к FGSM-атакам на свои данные, а модель 1 демонстрирует лучшую устойчивость к атакам по переносу. Для усиления защиты модели 2 целесообразно внедрить adversarial training и дополнительные методы регуляризации, чтобы уменьшить её восприимчивость к атакам.